In [1]:
#importing necessary libraries
import requests 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
from geopy.geocoders import Nominatim

In [2]:
#importing folium for maps, k-means for clustering, json for files, matplotlib
import json 
from pandas.io.json import json_normalize 

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

!pip -q install folium
import folium 

from sklearn.cluster import KMeans

In [3]:
CLIENT_ID = 'U4YRPZUOCCGOWOQ3QMBLJ0Q5OAJYUDZ2IVRTUFRPDV4IONGF' # Foursquare ID
CLIENT_SECRET = 'IRN5GFNMQH2B3TGYYS4ONVI3VQE2Q23TINXKDL0JORSRPE22' # Foursquare Secret
VERSION = '20200506'

In [4]:
URL = "https://en.wikipedia.org/wiki/List_of_London_boroughs"
a = requests.get(URL).text
b = BeautifulSoup(a,'lxml')

df_list = []
for items in b.find('table', class_= 'wikitable sortable').find_all('tr')[1::]:
    data = items.find_all(['td'])
    data1 = data[0]
    data8 = data[8]    
    try:
        borough_name = data1.get_text()
        borough_name = borough_name[0]
        ll = data8.get_text()
        lat_long = ll[2]
        latitude = lat_long[0]
        longitude = lat_long[1]
        df_list.append((borough_name, latitude, longitude))
    except IndexError:pass

In [5]:
URL = "https://en.wikipedia.org/wiki/List_of_London_boroughs"
a = requests.get(URL).text
b = BeautifulSoup(a,'lxml')

df_list = []
for items in b.find('table', class_= 'wikitable sortable').find_all('tr')[1::]:
    data = items.find_all(['td'])
    data1 = data[0]
    data8 = data[8]    
    try:
        borough_name = data1.get_text()
        borough_name = borough_name.split('[')
        borough_name = borough_name[0]
        borough_name = borough_name.strip()
        
        ll = data8.get_text()
        ll = ll.split('/')
        lat_long = ll[2]
        lat_long = lat_long.split('(')
        lat_long = lat_long[0]
        lat_long = lat_long.split(';')
        latitude = lat_long[0]
        latitude = latitude.strip()
        longitude = lat_long[1]
        longitude = longitude.strip()
        longitude = longitude.replace(u'\ufeff', '')
        latitude = float(latitude)
        longitude = float(longitude)

        df_list.append((borough_name, latitude, longitude))
    except IndexError:pass

In [6]:
df_boroughs = pd.DataFrame(df_list, columns=['Borough', 'Latitude' , 'Longitude'])

In [7]:
df_boroughs.shape

(32, 3)

In [8]:
df_boroughs.head()

,Borough,Latitude,Longitude
0,Barking and Dagenham,51.5607,0.1557
1,Barnet,51.6252,-0.1517
2,Bexley,51.4549,0.1505
3,Brent,51.5588,-0.2817
4,Bromley,51.4039,0.0198


In [9]:
df_boroughs.dtypes

Borough       object
Latitude     float64
Longitude    float64
dtype: object

In [10]:
df_boroughs.loc[df_boroughs['Borough'] == 'Newham']

,Borough,Latitude,Longitude
23,Newham,51.5077,0.0469


In [13]:
from geopy.geocoders import Nominatim 
address = 'London, UK'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of London City are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of London City are 51.5073219, -0.1276474.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


In [14]:
import folium 

map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough in zip(df_boroughs['Latitude'], df_boroughs['Longitude'], df_boroughs['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
fill_opacity=0.7).add_to(map_london)  
map_london

In [17]:
from urllib.request import urlopen
import re
URL = "https://en.wikipedia.org/wiki/List_of_areas_of_London"
a = requests.get(URL).text
b = BeautifulSoup(a,'lxml')

zip_codes = []
areas_list = []
href_links_list = []
for items in b.find('table', class_= 'wikitable sortable').find_all('tr')[1::]:
    data = items.find_all(['td'])
    data0 = data[0]
    area_name = data0.text

    data1 = data[1]
    data1 = data1.text
    borough = data1.split('[')
    borough_name = borough[0]
    data5 = data[5]
    code = data5.text
    code = code.strip()
    
    if borough_name == 'Newham':
        zip_codes.append(code)
        areas_list.append((borough_name,area_name,code))

                
for link in b.findAll('a', attrs={'href': re.compile("^https://tools.wmflabs.org")}):
            htext = link.text
            if htext in zip_codes:
                hlink = link.get('href')
                href_links_list.append((htext, hlink))

In [18]:
df_areas = pd.DataFrame(areas_list, columns=['Borough', 'Area', 'Zip_Code'])

In [19]:
df_areas.head()

,Borough,Area,Zip_Code
0,Newham,Beckton,TQ435815
1,Newham,Canning Town,TQ405815
2,Newham,Custom House,TQ408807
3,Newham,East Ham,TQ425835
4,Newham,Forest Gate,TQ405855


In [20]:
df_areas.columns

Index(['Borough', 'Area', 'Zip_Code'], dtype='object')

In [21]:
df_areas.shape

(14, 3)

In [22]:
df_links = pd.DataFrame(href_links_list, columns=['Zip_Code','href'])

In [23]:
df_links.columns

Index(['Zip_Code', 'href'], dtype='object')

In [24]:
df_links.shape

(15, 2)

In [25]:
df_links

,Zip_Code,href
0,TQ435815,https://tools.wmflabs.org/geohack/en/51.514205...
1,TQ405815,https://tools.wmflabs.org/geohack/en/51.514959...
2,TQ408807,https://tools.wmflabs.org/geohack/en/51.507695...
3,TQ425835,https://tools.wmflabs.org/geohack/en/51.532429...
4,TQ405855,https://tools.wmflabs.org/geohack/en/51.550902...
5,TQ435855,https://tools.wmflabs.org/geohack/en/51.550147...
6,TQ425855,https://tools.wmflabs.org/geohack/en/51.550401...
7,TQ391849,https://tools.wmflabs.org/geohack/en/51.545857...
8,TQ435795,https://tools.wmflabs.org/geohack/en/51.496234...
9,TQ405825,https://tools.wmflabs.org/geohack/en/51.523944...


In [27]:
c = df_links.columns.difference(df_areas.columns)

In [28]:
c

Index(['href'], dtype='object')

In [29]:
df_areas_links = pd.concat([df_areas, df_links[c]], axis=1)

In [30]:
df_areas_links.shape

(15, 4)

In [31]:
df_areas_links

,Borough,Area,Zip_Code,href
0,Newham,Beckton,TQ435815,https://tools.wmflabs.org/geohack/en/51.514205...
1,Newham,Canning Town,TQ405815,https://tools.wmflabs.org/geohack/en/51.514959...
2,Newham,Custom House,TQ408807,https://tools.wmflabs.org/geohack/en/51.507695...
3,Newham,East Ham,TQ425835,https://tools.wmflabs.org/geohack/en/51.532429...
4,Newham,Forest Gate,TQ405855,https://tools.wmflabs.org/geohack/en/51.550902...
5,Newham,Little Ilford,TQ435855,https://tools.wmflabs.org/geohack/en/51.550147...
6,Newham,Manor Park,TQ425855,https://tools.wmflabs.org/geohack/en/51.550401...
7,Newham,Maryland,TQ391849,https://tools.wmflabs.org/geohack/en/51.545857...
8,Newham,North Woolwich,TQ435795,https://tools.wmflabs.org/geohack/en/51.496234...
9,Newham,Plaistow,TQ405825,https://tools.wmflabs.org/geohack/en/51.523944...


In [32]:
df_areas_links = df_areas_links.dropna(how='any')

In [33]:
df_areas_links

,Borough,Area,Zip_Code,href
0,Newham,Beckton,TQ435815,https://tools.wmflabs.org/geohack/en/51.514205...
1,Newham,Canning Town,TQ405815,https://tools.wmflabs.org/geohack/en/51.514959...
2,Newham,Custom House,TQ408807,https://tools.wmflabs.org/geohack/en/51.507695...
3,Newham,East Ham,TQ425835,https://tools.wmflabs.org/geohack/en/51.532429...
4,Newham,Forest Gate,TQ405855,https://tools.wmflabs.org/geohack/en/51.550902...
5,Newham,Little Ilford,TQ435855,https://tools.wmflabs.org/geohack/en/51.550147...
6,Newham,Manor Park,TQ425855,https://tools.wmflabs.org/geohack/en/51.550401...
7,Newham,Maryland,TQ391849,https://tools.wmflabs.org/geohack/en/51.545857...
8,Newham,North Woolwich,TQ435795,https://tools.wmflabs.org/geohack/en/51.496234...
9,Newham,Plaistow,TQ405825,https://tools.wmflabs.org/geohack/en/51.523944...


In [43]:
geo_codes = []
for row in df_areas_links.itertuples():
    url = row.href
    zip_code = row.Zip_Code
    a = requests.get(url).text
    bb = BeautifulSoup(a,'lxml')
    
    for lat in bb.find('span',{'class':'latitude'}):
        latitude = lat
        latitude = float(latitude)
            
    for long in bb.find('span',{'class':'longitude'}):    
        longitude = long
        longitude = float(longitude)
        
    geo_codes.append((zip_code, latitude, longitude))

print(geo_codes)

[('TQ435815', 51.514206, 0.066634), ('TQ405815', 51.514959, 0.023429), ('TQ408807', 51.507696, 0.027431), ('TQ425835', 51.53243, 0.053041), ('TQ405855', 51.550902, 0.025024), ('TQ435855', 51.550148, 0.068263), ('TQ425855', 51.550401, 0.05385), ('TQ391849', 51.545857, 0.004608), ('TQ435795', 51.496234, 0.065821), ('TQ405825', 51.523945, 0.023828), ('TQ415795', 51.496738, 0.037029), ('TQ385845', 51.54241, -0.004196), ('TQ405837', 51.534728, 0.024306), ('TQ405837', 51.534728, 0.024306)]


In [44]:
df_geo_codes = pd.DataFrame(geo_codes, columns=['Zip_Code','Latitude','Longitude'])

In [45]:
df_geo_codes

,Zip_Code,Latitude,Longitude
0,TQ435815,51.514206,0.066634
1,TQ405815,51.514959,0.023429
2,TQ408807,51.507696,0.027431
3,TQ425835,51.532430,0.053041
4,TQ405855,51.550902,0.025024
5,TQ435855,51.550148,0.068263
6,TQ425855,51.550401,0.053850
7,TQ391849,51.545857,0.004608
8,TQ435795,51.496234,0.065821
9,TQ405825,51.523945,0.023828


In [46]:
df_areas.columns

Index(['Borough', 'Area', 'Zip_Code'], dtype='object')

In [47]:
df_areas.shape

(14, 3)

In [48]:
df_geo_codes.columns

Index(['Zip_Code', 'Latitude', 'Longitude'], dtype='object')

In [49]:
df_geo_codes.shape

(14, 3)

In [50]:
c = df_geo_codes.columns.difference(df_areas.columns)

In [51]:
c

Index(['Latitude', 'Longitude'], dtype='object')

In [52]:
Newham_borough = pd.concat([df_areas, df_geo_codes[c]], axis=1)

In [53]:
Newham_borough

,Borough,Area,Zip_Code,Latitude,Longitude
0,Newham,Beckton,TQ435815,51.514206,0.066634
1,Newham,Canning Town,TQ405815,51.514959,0.023429
2,Newham,Custom House,TQ408807,51.507696,0.027431
3,Newham,East Ham,TQ425835,51.532430,0.053041
4,Newham,Forest Gate,TQ405855,51.550902,0.025024
5,Newham,Little Ilford,TQ435855,51.550148,0.068263
6,Newham,Manor Park,TQ425855,51.550401,0.053850
7,Newham,Maryland,TQ391849,51.545857,0.004608
8,Newham,North Woolwich,TQ435795,51.496234,0.065821
9,Newham,Plaistow,TQ405825,51.523945,0.023828


In [54]:
Newham_borough = Newham_borough.rename(columns={'Area' :'Neighborhood'})

In [55]:
Newham_borough.drop(['Zip_Code'], axis=1, inplace=True)

In [56]:
Newham_borough.columns

Index(['Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

In [57]:
Newham_borough

,Borough,Neighborhood,Latitude,Longitude
0,Newham,Beckton,51.514206,0.066634
1,Newham,Canning Town,51.514959,0.023429
2,Newham,Custom House,51.507696,0.027431
3,Newham,East Ham,51.532430,0.053041
4,Newham,Forest Gate,51.550902,0.025024
5,Newham,Little Ilford,51.550148,0.068263
6,Newham,Manor Park,51.550401,0.053850
7,Newham,Maryland,51.545857,0.004608
8,Newham,North Woolwich,51.496234,0.065821
9,Newham,Plaistow,51.523945,0.023828


In [58]:
Newham_borough.dtypes

Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [59]:
address = 'Newham, London'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Newham are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinates of Newham are 51.52999955, 0.02931796029382208.


In [60]:
map_Newham = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(Newham_borough['Latitude'], Newham_borough['Longitude'], Newham_borough['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Newham)  
    
map_Newham

In [61]:
Newham_borough.shape

(14, 4)

In [62]:
Newham_borough.columns

Index(['Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

In [63]:
Newham_borough.loc[0, 'Neighborhood']

'Beckton'

In [64]:
Newham_borough.loc[0]

Borough           Newham
Neighborhood     Beckton
Latitude         51.5142
Longitude       0.066634
Name: 0, dtype: object

In [65]:
neighborhood_latitude = Newham_borough.loc[0, 'Latitude'] 
neighborhood_longitude = Newham_borough.loc[0, 'Longitude'] 

neighborhood_name = Newham_borough.loc[0, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Beckton are 51.514206, 0.066634.


In [66]:
LIMIT = 100 
radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret=\
       {}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=U4YRPZUOCCGOWOQ3QMBLJ0Q5OAJYUDZ2IVRTUFRPDV4IONGF&client_secret=       IRN5GFNMQH2B3TGYYS4ONVI3VQE2Q23TINXKDL0JORSRPE22&v=20200506&ll=51.514206,0.066634&radius=500&limit=100'

In [67]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb40a091835dd00289a6aaa'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Beckton',
  'headerFullLocation': 'Beckton, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 51.518706004500004,
    'lng': 0.07385150597103936},
   'sw': {'lat': 51.5097059955, 'lng': 0.05941649402896063}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e0c524018a89ad010688d2b',
       'name': 'East london Gymnastics Club',
       'location': {'lat': 51.514106774737556,
        'lng': 0.060155068624099396,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.514106774737556,
          'lng'

In [68]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [69]:
results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,East london Gymnastics Club,Gym / Fitness Center,51.514107,0.060155
1,Home Bargains,Discount Store,51.516805,0.062804
2,Lituanica,Grocery Store,51.516442,0.062927
3,Premier Inn London Beckton,Hotel,51.515115,0.061016
4,Matalan,Clothing Store,51.516004,0.062635


In [70]:
nearby_venues.shape

(9, 4)

In [71]:
nearby_venues['name'].value_counts()

Premier Inn London Beckton     1
Brewers Fayre                  1
East london Gymnastics Club    1
Home Bargains                  1
Dreams                         1
Beckton Retail Park            1
Beckton DLR Station            1
Lituanica                      1
Matalan                        1
Name: name, dtype: int64

In [72]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


In [73]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, 
            VERSION, lat, lng, radius, LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, lat, lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 
                             'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [74]:
Newham_venues = getNearbyVenues(names=Newham_borough['Neighborhood'],
                                   latitudes=Newham_borough['Latitude'],
                                   longitudes=Newham_borough['Longitude']
                                  )

In [75]:
Newham_venues.shape

(222, 7)

In [76]:
Newham_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Beckton,51.514206,0.066634,East london Gymnastics Club,51.514107,0.060155,Gym / Fitness Center
1,Beckton,51.514206,0.066634,Home Bargains,51.516805,0.062804,Discount Store
2,Beckton,51.514206,0.066634,Lituanica,51.516442,0.062927,Grocery Store
3,Beckton,51.514206,0.066634,Premier Inn London Beckton,51.515115,0.061016,Hotel
4,Beckton,51.514206,0.066634,Matalan,51.516004,0.062635,Clothing Store


In [77]:
Newham_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Beckton,9,9,9,9,9,9
Canning Town,4,4,4,4,4,4
Custom House,26,26,26,26,26,26
East Ham,13,13,13,13,13,13
Forest Gate,9,9,9,9,9,9
Little Ilford,4,4,4,4,4,4
Manor Park,5,5,5,5,5,5
Maryland,30,30,30,30,30,30
North Woolwich,22,22,22,22,22,22


In [78]:
print('There are {} uniques categories.'.format(len(Newham_venues['Venue Category'].unique())))

There are 102 uniques categories.


In [79]:
Newham_onehot = pd.get_dummies(Newham_venues[['Venue Category']], prefix="", prefix_sep="")

Newham_onehot['Neighborhood'] = Newham_venues['Neighborhood'] 

fixed_columns = [Newham_onehot.columns[-1]] + list(Newham_onehot.columns[:-1])
Newham_onehot = Newham_onehot[fixed_columns]

Newham_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,Art Gallery,Asian Restaurant,Bakery,Bar,Boat or Ferry,Bookstore,Boutique,...,Tapas Restaurant,Thai Restaurant,Theater,Toy / Game Store,Train Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar
0,Beckton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beckton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Beckton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Beckton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Beckton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
Newham_onehot.shape

(222, 103)

In [81]:
Newham_grouped = Newham_onehot.groupby('Neighborhood').mean().reset_index()
Newham_grouped.head()

,Neighborhood,Accessories Store,African Restaurant,Art Gallery,Asian Restaurant,Bakery,Bar,Boat or Ferry,Bookstore,Boutique,...,Tapas Restaurant,Thai Restaurant,Theater,Toy / Game Store,Train Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar
0,Beckton,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
1,Canning Town,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
2,Custom House,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.038462,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.038462
3,East Ham,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.076923,0.000000
4,Forest Gate,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.000000,0.000000


In [82]:
Newham_grouped.columns

Index(['Neighborhood', 'Accessories Store', 'African Restaurant',
       'Art Gallery', 'Asian Restaurant', 'Bakery', 'Bar', 'Boat or Ferry',
       'Bookstore', 'Boutique',
       ...
       'Tapas Restaurant', 'Thai Restaurant', 'Theater', 'Toy / Game Store',
       'Train Station', 'Turkish Restaurant', 'Video Game Store',
       'Vietnamese Restaurant', 'Warehouse Store', 'Wine Bar'],
      dtype='object', length=103)

In [83]:
Newham_grouped.shape

(14, 103)

In [ ]:
num_top_venues = 5
for hood in Newham_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Newham_grouped[Newham_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Beckton----
            venue  freq
0  Shopping Plaza  0.11
1  Clothing Store  0.11
2  Discount Store  0.11
3           Hotel  0.11
4             Pub  0.11


----Canning Town----
                       venue  freq
0                Gas Station  0.25
1                       Park  0.25
2          Convenience Store  0.25
3                       Café  0.25
4  Middle Eastern Restaurant  0.00


----Custom House----
              venue  freq
0             Hotel  0.19
1       Coffee Shop  0.08
2               Pub  0.08
3         Hotel Bar  0.04
4  Tapas Restaurant  0.04


----East Ham----


In [86]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [87]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Newham_grouped['Neighborhood']

for ind in np.arange(Newham_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Newham_grouped.iloc[ind, :], num_top_venues)

In [88]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beckton,Hotel,Furniture / Home Store,Pub,Clothing Store,Light Rail Station,Shopping Plaza,Discount Store,Grocery Store,Gym / Fitness Center,Fish & Chips Shop
1,Canning Town,Park,Convenience Store,Gas Station,Café,Fountain,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant
2,Custom House,Hotel,Pub,Coffee Shop,Harbor / Marina,Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Hotel Bar,Gym / Fitness Center
3,East Ham,Clothing Store,Optical Shop,Fast Food Restaurant,Pub,Coffee Shop,Electronics Store,Supermarket,Sandwich Place,Gym Pool,Park
4,Forest Gate,Grocery Store,Train Station,Bakery,Pub,Fast Food Restaurant,Fish & Chips Shop,Café,Wine Bar,Fountain,Doner Restaurant


In [89]:
Newham_grouped.head()

,Neighborhood,Accessories Store,African Restaurant,Art Gallery,Asian Restaurant,Bakery,Bar,Boat or Ferry,Bookstore,Boutique,...,Tapas Restaurant,Thai Restaurant,Theater,Toy / Game Store,Train Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar
0,Beckton,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
1,Canning Town,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
2,Custom House,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.038462,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.038462
3,East Ham,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.076923,0.000000
4,Forest Gate,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.000000,0.000000


In [90]:
kclusters = 5
Newham_grouped_clustering = Newham_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Newham_grouped_clustering)

kmeans.labels_[0:10]

array([1, 0, 1, 1, 0, 3, 4, 1, 1, 0], dtype=int32)

In [91]:
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [92]:
Newham_merged = Newham_borough

Newham_merged['Cluster Labels'] = kmeans.labels_

Newham_merged = Newham_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Newham_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Newham,Beckton,51.514206,0.066634,1,Hotel,Furniture / Home Store,Pub,Clothing Store,Light Rail Station,Shopping Plaza,Discount Store,Grocery Store,Gym / Fitness Center,Fish & Chips Shop
1,Newham,Canning Town,51.514959,0.023429,0,Park,Convenience Store,Gas Station,Café,Fountain,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant
2,Newham,Custom House,51.507696,0.027431,1,Hotel,Pub,Coffee Shop,Harbor / Marina,Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Hotel Bar,Gym / Fitness Center
3,Newham,East Ham,51.532430,0.053041,1,Clothing Store,Optical Shop,Fast Food Restaurant,Pub,Coffee Shop,Electronics Store,Supermarket,Sandwich Place,Gym Pool,Park
4,Newham,Forest Gate,51.550902,0.025024,0,Grocery Store,Train Station,Bakery,Pub,Fast Food Restaurant,Fish & Chips Shop,Café,Wine Bar,Fountain,Doner Restaurant


In [93]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
y = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(y)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Newham_merged['Latitude'], Newham_merged['Longitude'], Newham_merged['Neighborhood'], Newham_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [94]:
Newham_merged.loc[Newham_merged['Cluster Labels'] == 0, Newham_merged.columns[[1] + list(range(5, Newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Canning Town,Park,Convenience Store,Gas Station,Café,Fountain,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant
4,Forest Gate,Grocery Store,Train Station,Bakery,Pub,Fast Food Restaurant,Fish & Chips Shop,Café,Wine Bar,Fountain,Doner Restaurant
9,Plaistow,Grocery Store,Park,Vietnamese Restaurant,African Restaurant,Gym / Fitness Center,Bus Stop,Café,Pub,English Restaurant,Fish & Chips Shop


In [95]:
Newham_merged.loc[Newham_merged['Cluster Labels'] == 1, Newham_merged.columns[[1] + list(range(5, Newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beckton,Hotel,Furniture / Home Store,Pub,Clothing Store,Light Rail Station,Shopping Plaza,Discount Store,Grocery Store,Gym / Fitness Center,Fish & Chips Shop
2,Custom House,Hotel,Pub,Coffee Shop,Harbor / Marina,Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Hotel Bar,Gym / Fitness Center
3,East Ham,Clothing Store,Optical Shop,Fast Food Restaurant,Pub,Coffee Shop,Electronics Store,Supermarket,Sandwich Place,Gym Pool,Park
7,Maryland,Hotel,Pub,Café,Bus Stop,Indian Restaurant,Indie Movie Theater,Eastern European Restaurant,Dance Studio,Liquor Store,Grocery Store
8,North Woolwich,Hotel,Clothing Store,Pier,Plaza,Café,Pub,River,Italian Restaurant,Sculpture Garden,Shoe Store
10,Silvertown,Gym / Fitness Center,Theater,Museum,Construction & Landscaping,Café,Park,Italian Restaurant,Paintball Field,Go Kart Track,General Entertainment
11,Stratford,Pub,Clothing Store,Ice Cream Shop,Pizza Place,Juice Bar,Department Store,Sandwich Place,Cosmetics Shop,Coffee Shop,Shopping Mall


In [96]:
Newham_merged.loc[Newham_merged['Cluster Labels'] == 2, Newham_merged.columns[[1] + list(range(5, Newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Upton Park,Bakery,Convenience Store,Boutique,Bus Stop,Wine Bar,Furniture / Home Store,Eastern European Restaurant,Electronics Store,English Restaurant,Fast Food Restaurant
13,West Ham,Bakery,Convenience Store,Boutique,Bus Stop,Wine Bar,Furniture / Home Store,Eastern European Restaurant,Electronics Store,English Restaurant,Fast Food Restaurant


In [97]:
Newham_merged.loc[Newham_merged['Cluster Labels'] == 3, Newham_merged.columns[[1] + list(range(5, Newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Little Ilford,Burger Joint,Ice Cream Shop,Fried Chicken Joint,Business Service,Frozen Yogurt Shop,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Fast Food Restaurant


In [98]:
Newham_merged.loc[Newham_merged['Cluster Labels'] == 4, Newham_merged.columns[[1] + list(range(5, Newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Manor Park,Gym / Fitness Center,Turkish Restaurant,Asian Restaurant,Restaurant,Gas Station,Wine Bar,Fountain,Doner Restaurant,Donut Shop,Eastern European Restaurant
